In [ ]:
import os
import pandas as pd
import numpy as np

# Detect environment
KAGGLE_ENV = os.path.exists('/kaggle/input')

print(f"Running on Kaggle: {KAGGLE_ENV}")

if KAGGLE_ENV:
    print("Using Kaggle competition data")
    DATA_PATH = '/kaggle/input/histopathologic-cancer-detection/'
    
    # Load competition data
    train_df = pd.read_csv(f'{DATA_PATH}train.csv')
    test_df = pd.read_csv(f'{DATA_PATH}test.csv')
    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")
    
else:
    print("Running locally - using test data for development")
    # Create test data for local algorithm testing
    # You can modify this based on your competition's data structure
    test_array = [1, 4, 2, -2, 0, -1, 2, 3]
    test_sets = [[1, 2, 3, 4, 5, 10], [11, 23, 37, 48, 94, 152]]
    print("Local test data ready")

OSError: Could not find kaggle.json. Make sure it's located in /Users/hermanjustino/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/